### Processo de Extração de Entidades e Recuperação de Dados da Wikidata
Neste notebook serão executadas as ações de recuperação das entidades do corpus de redações e 
também, a recuperação de dados dessas entidades, a partir da wikidata, para assim gerar
uma lista de entidades e os seus possíveis alias.

In [ ]:
import pandas as pd
import wikidata
import json

In [ ]:
df_redacoes = pd.read_json("../ccscore/data/todas_redacoes_norm.json")

In [ ]:
####### TESTE #######


df_redacoes

In [ ]:
####### TESTE #######

num_red = 100
print(df_redacoes.loc[num_red, ["original"]][0])
print()
print(df_redacoes.loc[num_red, ["criteria"]][0])

In [ ]:
####### TESTE #######

num_red = 0
original = df_redacoes.loc[num_red, ["original"]][0]
fixed = df_redacoes.loc[num_red, ["fixed"]][0]
print(original)
print()
#print(fixed)

In [ ]:
import helper_dbpedia as dbpedia

In [ ]:
####### TESTE #######

entities = dbpedia.get_dbpedia_entries(original)

In [ ]:
####### TESTE #######

dbpedia.get_dbpedia_entries('aço')

In [ ]:
####### TESTE #######

import wikipedia

wikipedia.set_lang("pt")
wikipedia.search("aço")
page = wikipedia.page("aço")

In [ ]:
####### TESTE #######

dir(page)

atr_vals = [
page.categories,
page.content,
#page.coordinates,
page.html,
page.images,
page.links,
page.original_title,
page.pageid,
page.parent_id,
page.references,
page.revision_id,
page.section,
page.sections,
page.summary,
page.title,
page.url
]

print("\n".join([str(x) for x in atr_vals if not (x is None)]))
#"Elemento" in page.links

In [ ]:
####### TESTE #######

import requests

item_wiki="pr"   
#URL_REQUEST=f"https://pt.wikipedia.org/w/api.php?action=query&generator=links&format=json&redirects=1&prop=pageprops&gpllimit=50&ppprop=wikibase_item&titles={item_wiki}"
URL_REQUEST=f"https://pt.wikipedia.org/w/api.php?action=query&generator=links&format=json&redirects=1&prop=pageprops&gpllimit=50&ppprop=wikibase_item&titles={item_wiki}"

try:
    response = requests.get(URL_REQUEST)    
    response.raise_for_status()
except HTTPError as http_err:
    print(f'Erro HTTP: {http_err}') 
except Exception as err:
    print(f'Erro genérico: {err}')
else:
    print('Successo!')

    
#print(response.content)
dados_wiki_item = json.loads(response.content)
s_json = response.content

print(json.dumps(dados_wiki_item, indent=3))



#print(dados_wiki_item)

In [ ]:
import requests
from requests.exceptions import HTTPError
import json
import pydash

def get_wikipedia_page(item_wiki, show_raw=False):    
    URL_REQUEST=f"https://pt.wikipedia.org/w/api.php?action=query&prop=pageprops&format=json&titles={item_wiki}"
    try:
        response = requests.get(URL_REQUEST)    
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Erro HTTP: {http_err}') 
    except Exception as err:
        print(f'Erro genérico: {err}')
        
    dados_wiki_item = json.loads(response.content)

    if show_raw:
        print(json.dumps(dados_wiki_item, indent=3))

    wikibase_item = None        
    type_item='REDIRECT_PAGE'
    title_item = ""
    is_disambiguation_page = False
    is_redirect_page = True
    query = dados_wiki_item['query'] if 'query' in dados_wiki_item.keys() else None
    pages = query['pages'] if 'pages' in query.keys() else None
    for p in pages.values():
        for k,v in p.items():
            if k == 'pageprops':
                is_redirect_page = False
                if 'wikibase_item' in v.keys():
                    type_item='WIKIBASE_ITEM'
                    wikibase_item = v['wikibase_item']
                if 'disambiguation' in v.keys():
                    type_item='DISAMBIGUATION_PAGE'
                    is_disambiguation_page = True
            elif k == 'title':
                title_item = v
      
    return type_item, wikibase_item, title_item


def get_redirect_page(item_wiki, show_raw=False):
    URL_REQUEST=f"https://pt.wikipedia.org/w/api.php?action=query&&redirects&format=json&titles={item_wiki}"
    
    try:
        response = requests.get(URL_REQUEST)    
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Erro HTTP: {http_err}') 
    except Exception as err:
        print(f'Erro genérico: {err}')    

    dados_wiki_item = json.loads(response.content)

    if show_raw:
        print(json.dumps(dados_wiki_item, indent=3))

    #redirect_to = pydash.get(dados_wiki_item, 'query.redirects[0].to')
    redirects = pydash.get(dados_wiki_item, 'query.redirects')

    results = []
    if len(redirects) > 0:
        for r_item in redirects:
            redirect_to = r_item['to']
            results.append(
                            get_wikipedia_page(item_wiki=redirect_to,
                                               show_raw=show_raw)
                          )

    return results


def get_disambiguation_page(item_wiki, show_raw=False):
    URL_REQUEST=f"https://pt.wikipedia.org/w/api.php?action=query&generator=links&format=json&redirects=1&prop=pageprops&gpllimit=50&ppprop=wikibase_item&titles={item_wiki}"
    DESCARTE_QIDS = ['Q4167410',   # Wikipedia:Desambiaguação
                 'Q151',       # Wikcionário
                 'Q4167836',    # Wikimedia category                 
                ]  
    try:
        response = requests.get(URL_REQUEST)    
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'Erro HTTP: {http_err}') 
    except Exception as err:
        print(f'Erro genérico: {err}')    
        
    dados_wiki_item = json.loads(response.content)

    if show_raw:
        print(json.dumps(dados_wiki_item, indent=3))
        
    redirects = []
    
    redirects_from = pydash.get(dados_wiki_item, 'query.redirects.from')
    if not(redirects_from is None):
        redirects += redirects_from
    
    redirects_to = pydash.get(dados_wiki_item, 'query.redirects.to')
    if not(redirects_to is None):
        redirects += redirects_to
    
    redirects = set(redirects)

    pages = pydash.get(dados_wiki_item, 'query.pages')
    
    if show_raw:
        print(f"\n{50*'='}\n")
        print(json.dumps(pages, indent=3))
        print(f"\n{50*'='}\n")
        
    results = []
    if not(pages is None):
        for p_item in pages.values():
            p_item_title = p_item['title']
            p_wikibase_item = pydash.get(p_item, "pageprops.wikibase_item")

            # não quero redirects 
            if p_item_title in redirects:    
                continue
                
            # não quero itens vazios e itens da lista
            # de descarte
            if p_wikibase_item is None or \
               p_wikibase_item in DESCARTE_QIDS:
                continue
            
            # Recupera a pagina referente ao item
            tipo, item, title = get_wikipedia_page(p_item_title, show_raw)
            
            # Queremos apenas itens da wikidata, sem páginas
            # de redirecionamento e desambiguação
            if tipo == 'WIKIBASE_ITEM':
                results.append((tipo, item, title))
                

    return results
        
def get_wikipedia_data(item_wiki, show_raw=False):
    tipo, item, title = get_wikipedia_page(item_wiki, show_raw)
    
    result = []
    
    if tipo == 'WIKIBASE_ITEM':
        print("O item consultado é um 'item direto da wikipedia'.")        
        result = [ (tipo, item, title) ]
    elif tipo == 'REDIRECT_PAGE':    
        print("O item consultado é uma página de redirecionamento.")
        result_redir = get_redirect_page(item_wiki, show_raw)
        
        # As vezes a pagina de redirecionamento pode levar
        # para uma pagina de desambiguacao
        if len(result_redir) == 1 and \
           result_redir[0][0] == 'DISAMBIGUATION_PAGE':
            result = get_disambiguation_page(item_wiki, show_raw)
        else:
            result = result_redir                
        
    elif tipo == 'DISAMBIGUATION_PAGE':    
        print("O item consultado é uma página de desambiguação.")        
        result = get_disambiguation_page(item_wiki, show_raw)
    
    return result


In [ ]:
####### TESTE #######

#print(get_wikipedia_page("Paraná", show_raw=True))
print(get_wikipedia_page("Paint Rock", show_raw=True))


In [ ]:
####### TESTE #######

#results_wikidata = get_wikipedia_data("patrão", show_raw=True)
results_wikidata = get_wikipedia_data("legislativo", show_raw=False)

#if tipo == 'REDIRECT_PAGE':    
#    tipo, item, title = get_redirect_page("legislativo", show_raw=True)
    
for tipo,item,title in results_wikidata:    
    print(tipo)
    print()
    print(item)
    print()
    print(title)


In [ ]:
from urllib.error import HTTPError
from wikidata.client import Client

STR_PTBR = 'pt-br'
STR_PT = 'pt'
STR_EN = 'en'

def get_str_lang(item):
    if isinstance(item, dict):
        if STR_PTBR in item.keys():
            return STR_PTBR
        elif STR_PT in item.keys():
            return STR_PT
        elif STR_EN in item.keys():
            return STR_EN
        else:
            return None

def get_field_values(wikidata_item, item_field):
    str_lang = get_str_lang(wikidata_item.data[item_field])
    values_field = []
    if str_lang:
        if isinstance(wikidata_item.data[item_field][str_lang], dict):
            values_field = [
                             elem for k, elem in wikidata_item.data[item_field][str_lang].items() 
                                  if k == 'value'
                           ]
        elif isinstance(wikidata_item.data[item_field][str_lang], list):
            values_field = [
                             elem for x in wikidata_item.data[item_field][str_lang] 
                                      for k,elem in x.items() if k == 'value'
                           ]    
        
    return values_field

def get_wikidata_aliases(wikibase_item):
    
    aliases_pt_all = []
    
    aliases_ptbr = pydash.get(wikibase_item.data, 'aliases.pt-br')        
    if not(aliases_ptbr is None):
        aliases_pt_all += list({ x['value'] for x in aliases_ptbr })
       
    aliases_pt = pydash.get(wikibase_item.data, 'aliases.pt')        
    if not(aliases_pt is None):
        aliases_pt_all += list({ 
                                x['value'] for x in aliases_pt 
                                    if not(x['value'] in aliases_pt_all)
                               })

    return list(aliases_pt_all)
        
    
def get_wikidata_descriptions(wikibase_item):
    
    desc_pt_all = set([])
    
    desc_ptbr = pydash.get(wikibase_item.data, 'descriptions.pt-br.value')
    if not(desc_ptbr is None):
        desc_pt_all.add(desc_ptbr)
    
    desc_pt = pydash.get(wikibase_item.data, 'descriptions.pt.value')
    if not(desc_pt is None):
        desc_pt_all.add(desc_pt)
    
    return list(desc_pt_all)
    
def get_wikidata_labels(wikibase_item):
    labels_pt_all = set([])
    
    labels_ptbr = pydash.get(wikibase_item.data, 'labels.pt-br.value')        
    if not(labels_ptbr is None):
        labels_pt_all.add(labels_ptbr) 
    
    labels_pt = pydash.get(wikibase_item.data, 'labels.pt.value')    
    if not(labels_pt is None):
        labels_pt_all.add(labels_pt)
    
    return list(labels_pt_all)
    

def get_wikidata_info(wikibase_item):
    
    client_wikidata = Client()
    try:
        wikidata_item = client_wikidata.get(wikibase_item, load=True)
        
        labels_item = get_wikidata_labels(wikidata_item)
        descriptions_item = get_wikidata_descriptions(wikidata_item)    
        aliases_item = get_wikidata_aliases(wikidata_item)    
        
        return labels_item, descriptions_item, aliases_item
    
    except HTTPError as e:
        if e.code == 404:
            print(f"Item não encontrado na WikiData - erro {e.code}")        
        else: 
            print(f"Erro HTTP","\n", f"Erro {e.code}") 
      
    

In [ ]:
####### TESTE #######


c_wk = Client()

item_wkdt = c_wk.get("Q15499")

print(item_wkdt.attributes['labels']['pt'])
print(item_wkdt.attributes['descriptions']['pt'])
print(item_wkdt.attributes['aliases']['pt'])

print(f"descriptions: {get_wikidata_descriptions(item_wkdt)}")
print(f"labels: {get_wikidata_labels(item_wkdt)}")
print(f"aliases: {get_wikidata_aliases(item_wkdt)}")

In [ ]:
def get_wikientity_data(item_query, show_raw=False):
    wikipedia_data = get_wikipedia_data(item_query, show_raw)
    items = []
    for tipo, item_id, title in wikipedia_data:
        labels_item, descriptions_item, aliases_item = get_wikidata_info(item_id)        
        item_data = {'id': item_id,
                     'title': title,
                     'labels': labels_item,
                     'descriptions': " ".join(descriptions_item),
                     'aliases': aliases_item}
        items.append(item_data)
        
        if show_raw:            
            print(f"\nItem consultado: {title} - {item_id}\n")            
            print(item_data,"\n")
    return items

In [ ]:
####### TESTE #######
item_consulta = "aço"
get_wikientity_data(item_consulta, False)

In [ ]:
def get_line():
     with open(PATH_TODAS_REDACOES, 'r') as arq_redacoes:
            for linha in arq_redacoes:
                yield linha

In [ ]:
import itertools

def e_linha_titulo(linha):
    return "<ARQUIVO: " in linha or \
           linha == ""

PATH_TODAS_REDACOES="../ccscore/data/all_texts.txt"
def get_redacao():
    with open(PATH_TODAS_REDACOES, 'r') as arq_redacoes:        
        for key,group in itertools.groupby(arq_redacoes, e_linha_titulo):            
            if not key:
                yield "".join(list(group))
    

In [ ]:
# Gera uma lista com todas as redações

i = 0 
redacoes = []
for r in get_redacao():        
    redacoes.append(r)    

In [ ]:
####### TESTE #######

redacoes[1001]

In [ ]:
from pymongo import MongoClient
MONGO_IP_SERVER = "willbot"
MONGO_ROOT = "root"
MONGO_PASS = "Ktap1mb@!"
def get_mongo_client():
    try:
        mongoclient = MongoClient(MONGO_IP_SERVER, 
                              username=MONGO_ROOT, 
                              password=MONGO_PASS)    
        
        return mongoclient
    except:
        return None
    
def get_mongo_db(database, mongo_client=None):
    if mongo_client is None:
        mongo_client = get_mongo_client()
    
    return mongo_client[database]
                

def get_mongo_collection(collection_name, mongo_db):    
    dic_collection = None
    try:
        if collection_name not in mongo_db.list_collection_names():
            dic_collection = mongo_db.create_collection(collection_name)
        else:
            dic_collection = mongo_db[collection_name]        
    except Exception as e:
        print(f"Erro ao tentar criar a collection {collection_name}")
        print(str(e))

    return dic_collection

def insere_entidade_db(collection, entity_obj):  
    '''
    Insere um objeto entidade no MongoDB
    Esperar um objeto do tipo:
    r_jsonl = {'id': id_item,
                   'name': titulo,
                   'description': descricao,
                   'label': ""    
                    }
    '''
    qtd_ents = collection.count_documents({'name': entity_obj['name']})
    
    if qtd_ents == 0:    
        try:
            id_inserted = collection.insert_one(entity_obj).inserted_id

            return id_inserted
        except Exception as e:
            print(f"Erro ao tentar inserir a entidade {entity_obj['name']}")
            print(str(e))
    else:
        return None
    
    
def get_entity_by_qid(collection, id_entity):    
    q_entity = collection.find_one({'id': id_entity})
    
    return q_entity


def reajusta_probs(num_items):
    '''
    Ajusta os valores de probabilidade, distribuindo
    os valores de acordo com a quantidade de itens
    '''
    l_r = num_items*[round(1/num_items,2)]
    
    if sum(l_r) < 1.0:
        l_r[0] += (1 - sum(l_r))
    elif sum(l_r) > 1.0:
        l_r[-1] -= (sum(l_r)-1)
    
    return l_r
    
def insere_alias_db(collection, alias_obj):
    '''
    Insere um objeto alias no MongoDB
    Esperar um objeto do tipo:
    a_jsonl = {'alias': titulo,
                   'entities': [id_item],
                   'probabilities': [1.0]}
    '''
    # Verifica se o alias inserido já existe    
    qtd_alias = collection.count_documents({'alias': alias_obj['alias']})
    
    if qtd_alias == 0:  # não existe no db                
        try:
            id_inserted = collection.insert_one(alias_obj).inserted_id

            return id_inserted
        except Exception as e:
            print(f"Erro ao tentar inserir aliases na base. \n{str(e)}")
            return None
    else:   # substitui             
        doc_alias = collection.find_one({'alias': alias_obj['alias']})
        qt_entities = len(doc_alias['entities'])
        new_entity_list = list(set(doc_alias['entities']+alias_obj['entities']))
        new_qt_entity_list = len(new_entity_list)
        if qt_entities < new_qt_entity_list: # alguma entitidade foi acrescida?
            new_probs = reajusta_probs(new_qt_entity_list)
            result = collection.update_one(
                            {"_id" : doc_alias["_id"]},
                            {
                                "$set": {'entities': new_entity_list,
                                    'probabilities': new_probs}
                            },
                            upsert=True)
            if not result.acknowledged:
                print(f"Erro ao tentar atualizar o documento {doc_alias['_id']}")
        else:  # não há nada de novo para acrescentar
            return None


def get_alias_by_name(collection, name_alias):
    mongo_alias = collection.find_one({'alias': name_alias})
    
    if not (mongo_alias is None):        
        return mongo_alias
    else:
        return None

    

In [ ]:
kb_mongo = get_mongo_db("kb")
raw_kb = get_mongo_collection("raw_kb", kb_mongo)
entidades_kb = get_mongo_collection("entidades_kb", kb_mongo)
alias_kb = get_mongo_collection("alias_kb", kb_mongo)

for num_redacao, redacao in enumerate(redacoes[:10]):
    if len(redacao) < 5:
        continue

    print()
    print(50*"=")
    print(f"Processando redação {num_redacao}")
    print(50*"=")
    entidades_dbpedia = dbpedia.get_dbpedia_entries(redacao)

    for t_ent, d_ent in entidades_dbpedia.items():
        try:
            print(f"Processando entidade {t_ent}")
            
            #qtd_ents = raw_kb.count_documents({'dbpedia_text': d_ent['dbpedia_text']})
            qtd_ents = raw_kb.count_documents({'dbpedia_text': d_ent['raw_text']})
            
            if qtd_ents > 0: # inserir apenas um documento por item
                print(f"Entidade {t_ent} já existe na base de dados.")
            else:                 
                id_inserido = raw_kb.insert_one(d_ent).inserted_id
                print(f"Inserido id {id_inserido}")
            
                #wikidata_items = get_wikientity_data(d_ent['dbpedia_text'])
                wikidata_items = get_wikientity_data(d_ent['raw_text']) 
                for item_data in wikidata_items:
                    print(f"\n\nitem_data {item_data}\n\n")
                    item_data_id   = item_data['id']
                    item_data_name = item_data['title']
                    item_data_desc = item_data['descriptions']
                    item_data_labels = item_data['labels']
                    item_data_aliases = item_data['aliases']

                    #print(item_data_id)
                    #print(item_data_name)
                    #print(item_data_desc)
                    #print(item_data_labels)
                    #print(item_data_aliases)

                    # Insere entidade 
                    r_jsonl = {
                               'id': item_data_id,
                               'name': item_data_name,
                               'description': item_data_desc,
                               'label': "",  # este campo é um indicador da base de conhecimento 
                                             # diferente do label retornado da wikidata
                              }
                    insere_entidade_db(entidades_kb, r_jsonl)

                    # Insere um alias equivalente a entidade
                    a_jsonl = {
                               'alias': item_data_name,
                               'entities': [item_data_id],
                               'probabilities': [1.0]
                              }
                    insere_alias_db(alias_kb, a_jsonl)

                    # Se o titulo for diferente dos labels da wikidata
                    # insere um alias equivalente            
                    #if any([title.lower() != l.lower() for l in labels_item]):

                    for alias_l in [l for l in item_data_labels if item_data_name.lower() != l.lower()]:
                        a_jsonl = {
                                   'alias': alias_l,
                                   'entities': [item_data_id],
                                   'probabilities': [1.0]
                                  }
                        insere_alias_db(alias_kb, a_jsonl)

                    # Insere os alias
                    for alias_l in item_data_aliases:
                        a_jsonl = {
                                   'alias': alias_l,
                                   'entities': [item_data_id],
                                   'probabilities': [1.0]
                                  }            
                        insere_alias_db(alias_kb, a_jsonl)
                
        except Exception as e:
            print(f"Erro processando entidade {t_ent}")
            print(f"Erro: {str(e)}")

In [ ]:
t = "titulo1"
labels_t = ["Titulo", "Titulo1"]

any([x.lower() == t.lower() for x in labels_t])

In [ ]:
len(redacoes[167])

In [ ]:
import multiprocessing 

GLOBALLOCK = multiprocessing.Lock()   

def worker_processa_redacao(l_redacoes):
    
    GLOBALLOCK.acquire()
    
    kb_mongo = get_mongo_db("base_conhecimento")
    raw_kb = get_mongo_collection("raw_kb", kb_mongo)
    entidades_kb = get_mongo_collection("entidades_kb", kb_mongo)
    alias_kb = get_mongo_collection("alias_kb", kb_mongo)
    
    GLOBALLOCK.release()
    
    for redacao in l_redacoes:    
        print()
        print(50*"=")
        print(f"Processando redação ...")
        print(50*"=")
        entidades_dbpedia = dbpedia.get_dbpedia_entries(redacao)

        for t_ent, d_ent in entidades_dbpedia.items():
            try:
                print(f"Processando entidade {t_ent}")

                id_inserido = raw_kb.insert_one(d_ent).inserted_id
                print(f"Inserido id {id_inserido}")

                tipo, item_id, title = get_wikipedia_page(d_ent['dbpedia_text'])
                wikidata_item = get_wikidata_aliases(item_id)            
                labels_item   = get_field_values(wikidata_item, 'labels')    
                descriptions_item = " ".join(get_field_values(wikidata_item, 'descriptions'))
                aliases_item = get_field_values(wikidata_item, 'aliases')

                item_data = {'id': item_id,
                             'title': title,
                             'labels': labels_item,
                             'descriptions': descriptions_item,
                             'aliases': aliases_item}

                # Insere entidade 
                r_jsonl = {
                           'id': item_id,
                           'name': title,
                           'description': descriptions_item,
                           'label': "",  # este label é um indicador da base de conhecimento 
                          }
                
                ####################
                GLOBALLOCK.acquire()
                ####################    
                
                insere_entidade_db(entidades_kb, r_jsonl)

                # Insere um alias equivalente a entidade
                a_jsonl = {
                           'alias': title,
                           'entities': [item_id],
                           'probabilities': [1.0]
                          }
                insere_alias_db(alias_kb, a_jsonl)

                # Se o titulo for diferente dos labels da wikidata
                # insere um alias equivalente
                if not(title in labels_item):
                    for alias_l in labels_item:
                        a_jsonl = {
                                   'alias': alias_l,
                                   'entities': [item_id],
                                   'probabilities': [1.0]
                                  }            
                        insere_alias_db(alias_kb, a_jsonl)


                # Insere os alias
                for alias_l in aliases_item:
                    a_jsonl = {
                               'alias': alias_l,
                               'entities': [item_id],
                               'probabilities': [1.0]
                              }            
                    insere_alias_db(alias_kb, a_jsonl)
                    
                ####################    
                GLOBALLOCK.release()
                ####################
                
            except Exception as e:
                print(f"Erro processando entidade {t_ent}")
                print(f"Erro: {str(e)}")

In [ ]:
num_procs = 20
n = len(redacoes)//num_procs
chunks_redacoes = [redacoes[i * n:(i + 1) * n] for i in range((len(redacoes) + n - 1) // n )]

In [ ]:
len(chunks_redacoes)
n

In [ ]:
p = multiprocessing.Pool(num_procs)
p.map(worker_processa_redacao, chunks_redacoes)
p.close()